In [1]:
import os
import pickle
import matplotlib.pyplot as plt
import sys
import os

sys.path.append(os.path.abspath(".."))  # Ensure it points to your project root

from van_code.nn import NeuralVANMultilevel_block_wise
from van_code.ising import ising_energy,analytical_solution,local_ising_energy
from van_code.montecarlo import *
from van_code.utils import *
from van_code.obs import Obs


ModuleNotFoundError: No module named 'autograd'

In [3]:
import autograd

ModuleNotFoundError: No module named 'autograd'

In [ ]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    default_dtype_torch=torch.float32
elif torch.cuda.is_available():
    device = 'cuda'
else:
    device = torch.device("cpu")
print(f'You are using device={device}.')

-------------
-------------
# Model and parameters
Our multilevel architecture uses different blocks of Autoregressive neural networks which are based on the VAN architecture by [Wu et al. (2019)](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.122.080602).

In [ ]:
# This cell defines important parameters for the simulation

Lc=2 # defines the coarser lattice
beta=0.44 # defines the beta value (inverse of the temperature)

# VAN parameters
net_depth = 3
net_width= 16
half_kernel_size =6

# Conditional VAN parameters
# The i-th element of the list correspond to the parameters associated to the i-th block in the multilevel

hidden_size=[[16]] * 10 # Hidden size of each CondVAN
kernel_size=[[5,3]] * 10  # Kernel of CondVAN

nlevels = 2 # Defines the number of blocks (upsamling step) in the multilevel. Every block doubles the lattice. Example: if the coarser is 2x2 after 3 levels we have 16x16
hb_last = True # Whether to use heatbath to sample unbiased configuration from the last ARNN

pretrained = True  # Wheter to use warm-restarts in training (see paper: https://openreview.net/forum?id=YcUV5apdlq)


In [ ]:
van_hyp={
    'net_depth':net_depth,
    'net_width': net_width,
    'half_kernel_size':half_kernel_size,
    'bias':False,'z2':False,
    'res_block':True,
    'x_hat_clip':False,
    'final_conv':True,
    'epsilon':1.e-8,
    'device':device
}

block_net_hyp={
    'hidden_size':hidden_size,
    'kernel_size':kernel_size,
    'epsilon':1e-7,
    'level':0,
    'device':device
}

In [ ]:
# Building the model

model = NeuralVANMultilevel_block_wise(
    Lc,
    van_hyp,
    block_net_hyp,
    nlevels,
    hb_last,
    ising_energy,
    local_ising_energy,
    beta,
    device
)
Lf = model.Lf
#print(model)

print(f'\n\n=======================================================================================================================================================\n')
print(f'You will start from coarser lattices of shape {Lc}x{Lc} and sample finer lattices of resolution {model.Lf}x{model.Lf} using {nlevels} multilevel steps.')
print(f'\n=======================================================================================================================================================\n')

In [ ]:
# Define default parameters for optimizer and scheduler

optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.92, patience=1000, min_lr=1e-07)

# Define additional training parameters. Arbitrary values can be chosen
# the values chosen below are optimized for the purpose of this demo.

bs = [16] * (nlevels+1)
nepochs = 2 * [500] * nlevels + [500]
print_freq = 10
lr = 2 * [0.001] + [0.001] * (nlevels-1)

In [ ]:
# Choose main path to store model and logs
main_path=''+'../data/'

# Creates model identifier with corresponding params
# '/Lf'+'_beta'+'_nblocks'+'_PCNNdepth'+'_width'+'_half_ker'+'_CCNNhs'+'_ks'

model_id = f'{str(model.Lf)}_{str(beta)}_{str(nlevels)}_{str(net_depth)}_{str(net_width)}_{str(half_kernel_size)}_{str(hidden_size[0][0])}_{str(kernel_size[0])}'

lc_path = main_path + 'training/' + model_id
h_path = lc_path + '_history.log' # Path to Training history

## Dict of paths to store different results

res_paths = {
    'weights': main_path+ 'model/'+ model_id +'.chckpnt', # models' weights
    'sim' : main_path + 'results/' + model_id + '_measures.log',  # Simulations logging
    'sim_md' : main_path + 'results/' + model_id + '_measures_modedrop.log',  # Simulations mode dropping
    'cluster' : main_path + 'results/' + model_id + '_measuresCluster.log',  # Simulations
    'dict_hist' : lc_path + '_historyDict.pkl',  # dictionary learning curve
    'mh' : main_path+'results/'+ model_id +'_measuresIMH.log' # neural MCMC sampling results
}


-------------
-------------
# Training

In [ ]:
# Load model if checkpoint exists, otherwise train a new model.

if os.path.exists(res_paths['weights']):
    print(f'\n============================================================================================================================\n')
    print(f"Loading model {res_paths['weights']}.")
    print(f'\n============================================================================================================================\n')
    load(model, optimizer, res_paths['weights'])
    with open(res_paths['dict_hist'], 'rb') as handle:
        history= pickle.load(handle)
    print(f'\n============================================================================================================================\n')
    print(f"Loading successful!")
    print(f'\n============================================================================================================================\n')

else:
    print(f'\n============================================================================================================================\n')
    print(f"Nothing to load. Training of new model... ")
    print(f'\n============================================================================================================================\n')

    history=model.train(nepochs, bs, lr, print_freq, h_path, pretrained, on_file=True)
    save(model, optimizer, res_paths['weights'])
    write(history, lc_path)

    print(f'\n============================================================================================================================\n')
    print(f"Training successful!")
    print(f'\n============================================================================================================================\n')


In [ ]:
bins = 10 # binning to plot training history.

if history:
    plt.plot(np.asarray(history['ESS']).reshape(int((nepochs[0]*nlevels+nepochs[-1])/bins),bins).mean(-1))
    plt.ylabel('ESS')

-------------
-------------
# Analysis: Reverse Metrics
This sections includes metrics which require samples from the model, e.g., $s\sim q_\theta$ where $q_\theta$ is the autoregressive neural network.
The resulting script `res_paths['sim']` will store:

`Loss, FreeEn, ESS_rev, InternalEn, Absmag`

In [ ]:
model.eval()
w,E,m,t = model.sample_n_OBS(100,1000)

In [ ]:
gamma_analysis(w, res_paths['sim']) # Stores data: Loss, FreeEn, ESS_rev
_=gamma_analysis_OBS(E, w, res_paths['sim']) # Stores data: Internal Energy
_=gamma_analysis_OBS(np.abs(m), w, res_paths['sim']) # Stores data: Absolute magnetization
print(f"Results saved at: {res_paths['sim']}")

-------------
-------------
# Analysis: Forward metrics
> Note to user:
> - This only works for small lattices. As this is just a demo, configurations for larger lattices (or different $\beta$) are not provided and users have to generate reference configurations by themselves. Once configurations are available the pipeline below can be used.
> - Ensure to unzip the configurations before running the code below.


This sections includes metrics which require samples from the true distribution, e.g., $s\sim p$ where $p$ is the target Boltzmann distribution.
Such samples can be obtained with standard methods such as the Cluster method. For the sake of this demo, configs sampled with cluster method are stored in `data/config/Ising_data_nx16_beta0.4400000000_data1000000.dat`.

The script `res_paths['sim_md']` will store:

`Loss_rev, FreeEn_rev, ESS_rev, Loss_fwd, FreeEn_fwd, ESS_fwd, mode_dropping_est,  InternalEn, Absmag`

The script `res_paths['cluster']` will store analysis performed on samples from Cluster method, i.e., $s\sim p$:

`Loss, FreeEn, ESS_rev, InternalEn, Absmag`


In [ ]:
ndat = 1000000
path_ising = main_path + f'config/Ising_data_nx{model.Lf}_beta0.4400000000_data{ndat}.dat'

In [ ]:
# Loads data for 16x16
# N.B. This might take some time.
data = np.genfromtxt(path_ising).reshape(-1, Lf, Lf)

In [ ]:
cluster_analysis(data, ising_energy, beta, res_paths['cluster']) # U, tau |m|, tau |m|
print(f"Results saved at: {res_paths['cluster']}")

In [ ]:
wf = model.sample_from_MCMC(data, 1000)

In [ ]:
gamma_analysis_modedrop(w, wf, res_paths['sim_md']) # Stores data: Loss, betaF, ESS, Floss, FbetaF, FESS, modedrop, U, absmag
print(f"Results saved at: {res_paths['sim_md']}")

-------------
-------------
# Independent Metropolis-Hastings

The cells below run Neural MCMC using a Metropolis Hastings accept-reject steps to unbias the samples drawn from the model as proposed in [Nicoli et al. Phys. Rev. E (2020)](https://link.aps.org/accepted/10.1103/PhysRevE.101.023304). A cluster analysis is performed on the accepted samples and results are saved in `res_paths['mh']`.

In [ ]:
model.eval()

In [ ]:
ensemble=make_mcmc_ensemble(model,100, 1000,model.device)

In [ ]:
print(f"Acceptance rate: {np.asarray(ensemble['accepted']).mean().item()}")

In [ ]:
 cluster_analysis(np.asarray(ensemble['x']).reshape((-1, Lf, Lf) ), model.energy, model.beta, res_paths['mh']) #U, tau |m|, tau |m|
print(f"Results saved at: {res_paths['mh']}")    